In [10]:

import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

from transformers import T5ForConditionalGeneration, T5Tokenizer, AdamW, get_linear_schedule_with_warmup
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
import pickle

data_dir = "/media/zihao/New Volume1/UMASS/685_e/github/Zihao_branch/data/Danmu_byt5/pkl"
type_path = 'train'

token_file_path = os.path.join(data_dir, type_path, 'danmu_token_single_label_'+type_path+'.pkl')
dist_file_path = os.path.join(data_dir, type_path, 'danmu_token_single_label_'+type_path+'.pkl')

In [13]:
token_file = open(token_file_path, "rb")
dist_file = open(dist_file_path, "rb")


max_len = 512
tokenizer = T5Tokenizer.from_pretrained('t5-base')
inputs = []
targets = []

In [14]:
tokens = pickle.load(token_file)
labels = pickle.load(dist_file)

REPLACE_NO_SPACE = re.compile("[.;:!\'?,\"()\[\]]")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

for ind, token in enumerate(tokens):
    text = token

    line = text.strip()
    line = REPLACE_NO_SPACE.sub("", line) 
    line = REPLACE_WITH_SPACE.sub("", line)
    line = line + ' </s>'

    target = labels[ind] + " </s>"

    # tokenize inputs
    tokenized_inputs = tokenizer.batch_encode_plus(
        [line], max_length=max_len, pad_to_max_length=True, return_tensors="pt"
    )
    # tokenize targets
    tokenized_targets = tokenizer.batch_encode_plus(
        [target], max_length=2, pad_to_max_length=True, return_tensors="pt"
    )

    inputs.append(tokenized_inputs)
    targets.append(tokenized_targets)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/zihao/anaconda3/envs/py37/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2217: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/home/zihao/anaconda3/envs/py37/lib/python3.7/site-packages/transformers/models